In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from itertools import islice
from urllib.parse import urlparse
import ast


In [2]:
# Load filtered an streamed FineWeb 2 dataset
dataset = load_dataset("HuggingFaceFW/fineweb-2", split="train", data_dir="data/dan_Latn", streaming=True)

In [3]:
def get_batches(ds, batch_size):
    iterator = iter(ds)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


def domain_distribution(df):
    if "domain" not in df.columns:
        raise ValueError("DataFrame does not contain a 'domain' column.")
    
    distribution = df["domain"].value_counts().sort_values(ascending=False)
    return distribution

def save_text_label(text, label, path="fineweb2_data.csv"):
    try:
        df = pd.read_csv(path)
        df["label"] = df["label"].apply(ast.literal_eval)  # convert label column from str to list
    except FileNotFoundError:
        df = pd.DataFrame(columns=["text", "label"])

    match = df[df["text"] == text]

    if not match.empty:
        idx = match.index[0]
        labels = df.at[idx, "label"]
        if label not in labels:
            labels.append(label)
            df.at[idx, "label"] = labels
    else:
        new_row = pd.DataFrame([{"text": text, "label": [label]}])
        df = pd.concat([df, new_row], ignore_index=True)

    df.to_csv(path, index=False)

In [4]:
Batch_Size = 1000
Batch_Iterations = 20
Remove_Domains = ["www.texaspoker.dk"]

batch_generator = get_batches(dataset, batch_size=Batch_Size)

# Initialize storage
all_records = pd.DataFrame()
all_domains = []

# Process in batches
for i in range(Batch_Iterations):
    print(f"Batch {i+1} of {Batch_Iterations}")
    batch = next(batch_generator)

    records = [{
        "text": example["text"],
        "minhash_cluster_size": example.get("minhash_cluster_size"),
        "url": example.get("url")
    } for example in batch]

    temp_Records = pd.DataFrame(records)
    temp_Records["domain"] = temp_Records["url"].apply(lambda url: urlparse(url).netloc if url else None)

    # Filter out domains
    filtered_Records = temp_Records[~temp_Records["domain"].isin(Remove_Domains)].reset_index(drop=True)

    all_records = pd.concat([all_records, filtered_Records], ignore_index=True)
    all_domains.extend(filtered_Records["domain"].unique())



    

Batch 1 of 20
Batch 2 of 20
Batch 3 of 20
Batch 4 of 20
Batch 5 of 20
Batch 6 of 20
Batch 7 of 20
Batch 8 of 20
Batch 9 of 20
Batch 10 of 20
Batch 11 of 20
Batch 12 of 20
Batch 13 of 20
Batch 14 of 20
Batch 15 of 20
Batch 16 of 20
Batch 17 of 20
Batch 18 of 20
Batch 19 of 20
Batch 20 of 20


In [7]:
print(domain_distribution(all_records))
print("Total Domains:",len(all_domains))

domain
da.wikipedia.org                                                        1039
www.dagens.dk                                                            706
ekstrabladet.dk                                                          388
www.andersen.sdu.dk                                                      375
www.bt.dk                                                                330
www.agoda.com                                                            319
tipsed.com                                                               253
www.mx.dk                                                                241
www.tripadvisor.dk                                                       203
www.kayak.dk                                                             189
www.jetcost.dk                                                           186
www.wimdu.dk                                                             181
www.avisen.dk                                                        

In [7]:
def browse_samples_by_domain(all_domains, all_records):
    pd.set_option('display.max_colwidth', 100)  # limit text preview
    pd.set_option('display.max_rows', False)    # show all rows if needed

    for domain in all_domains:
        domain_samples = all_records[all_records["domain"] == domain]

        print(f"\n--- Domain: {domain} ---")
        if domain_samples.empty:
            print("No samples available.")
        else:
            sample = domain_samples.sample(n=1).iloc[0]
            display(sample)
            print(f"Sample text:\n{sample['text']}\n")

        user_input = input("Press ENTER to continue or 'q' to quit: ").strip().lower()
        if user_input == 'q':
            print("Exiting...")
            break

In [ ]:
browse_samples_by_domain(all_domains, all_records)

In [ ]:
# 6, good, 31, good, 183, good, 1490 good, 2509, good, 164, good kinda, 198, good, 

In [11]:
check_domain = "europa.eu"

# Show all rows
pd.set_option('display.max_rows', None)

# Limit text column width (e.g. to 100 characters)
pd.set_option('display.max_colwidth', 500)

# Filter and display
all_records[all_records["domain"] == check_domain]

,text,minhash_cluster_size,url,domain
151,"Underretning om nationale foranstaltninger, der fraviger princippet om frie varebevægelser\nDet er lettere at løse problemer, der underminerer princippet om frie varebevægelser, hvis de nationale foranstaltninger, der hindrer frie varebevægelser, er gennemskuelige. Nærværende beslutning indfører en procedure, hvorved medlemsstaterne forpligtes til at underrette Kommissionen om nationale foranstaltninger, der fraviger princippet om frie varebevægelser. En sådan procedure bidrager til at løse ...",13,http://europa.eu/legislation_summaries/internal_market/single_market_for_goods/technical_harmonisation/l11040_da.htm,europa.eu
737,"Udstationering i udlandet\nOpdateret 03/2013\nHvis din virksomhed har en aftale med en forretningspartner i et andet EU-land, og du bliver nødt til at sende personale dertil i en begrænset periode i forbindelse med aftalen, skal du overholde reglerne for udstationering af ansatte. Reglerne gælder også, hvis du sender en ansat til en virksomhed, du selv ejer, i et andet EU-land. I begge tilfælde skal der være et ansættelsesforhold mellem dig og den udstationerede.\nReglerne om udstationering ...",60,http://europa.eu/youreurope/business/staff/posting-abroad/index_da.htm,europa.eu
773,"Glossar\nArtikel 71-Udvalget (afsnit V i TEUF)\nDet stående Artikel 71-Udvalg (TEUF-traktaten) er en arbejdsgruppe i Rådet, hvor de andre EU-organer og -agenturer også kan deltage. Det er Rådets opgave at koordinere medlemsstaternes operationelle samarbejde inden for frihed, sikkerhed og retfærdighed.\nDet stående udvalg understøtter også Den Udenrigs- og Sikkerhedspolitiske Komité i rådgivningen af Rådet med henblik på gennemførelse af EU's solidaritetsbestemmelse.\nSe også:",27,http://europa.eu/legislation_summaries/glossary/coordinating_committee_da.htm,europa.eu
960,"Tilgængelighedsværktøjer\nVærktøjer\nSprogvælger\nDanmark overtog formandskabet for Rådet for De Europæiske Fællesskaber. Det indre marked trådte i kraft.\nTiltrædelsesforhandlinger med Østrig, Finland og Sverige indledtes i Bruxelles. Fællesskabet og Rumænien undertegnede en Europaaftale om handelsanliggender.\nFællesskabet og Bulgarien undertegnede en Europaaftale om handelsanliggender.\nTiltrædelsesforhandlingerne med Norge indledtes i Luxembourg.\nKommissionen vedtog en grønbog om erstat...",33,http://europa.eu/about-eu/eu-history/1990-1999/1993/index_da.htm,europa.eu
1220,We are migrating the content of this website during the first semester of 2014 into the new EUR-Lex web-portal. We apologise if some content is out of date before the migration. We will publish all updates and corrections in the new version of the portal.\nDo you have any questions? Contact us.\nBeskyttelse af Europa mod storstilede cyberangreb og sammenbrud\nDen søgte information findes ikke på det valgte sprog.\nKlik på et af følgende links for at få adgang til informationer på et af de fo...,11,http://europa.eu/legislation_summaries/information_society/internet/si0010_da.htm,europa.eu
1431,We are migrating the content of this website during the first semester of 2014 into the new EUR-Lex web-portal. We apologise if some content is out of date before the migration. We will publish all updates and corrections in the new version of the portal.\nDo you have any questions? Contact us.\nFødevarer\nEU sætter pris på sin gastronomiske arv og får forbrugerne til at blive opmærksomme på fødevarernes kvalitet. Med det for øje tager EU-retten økologisk landbrug og kvalitetsprodukter i for...,26,http://europa.eu/legislation_summaries/agriculture/food/index_da.htm,europa.eu
1479,"We are migrating the content of this website during the first semester of 2014 into the new EUR-Lex web-portal. We apologise if some content is out of date before the migration. We will publish all updates and corrections in the new version of the portal.\nDo you have any questions? Contact us.\nGrundlæggende rettigheder og medborgerskab (2007-2013)\nVed denne afgørelse fastlægg

In [49]:
all_records.iloc[7205]["text"] # 2888, 2988, 1211

'Kort over den globale luftforurening\nDet fremgår tydeligt af dette billede, hvilke lokaliteter i verden, som er mest plaget af luftforurening. Billedet er baseret på 18 måneders observationer fra den europæiske miljøsatellit Envisat.\nKortet viser fordelingen af forureningen med kvælstofoxider i atmosfæren. Man kan se, at der er høje koncentrationer af denne type forurening knyttet til større byer tværs over hele Nordamerika og Europa såvel som andre steder, som f.eks. Mexico City i Mellemamerika og sydafrikanske kulfyrede kraftværker i Sydafrika, der er placeret i nærheden af hinanden i den østlige del af landets Highveld-højslette.\n"Desuden findes der en meget høj koncentration over det nordøstlige Kina. Også tværs over det sydøstlige Asien og store dele af Afrika kan der ses kvælstofdioxid produceret af afbrænding af biomasse. Spor fra skibe er synlige visse steder: Se på Det Røde Hav og Det Indiske Ocean mellem Indiens sydspids og Indonesien. Skibes skorstene sender disse steder

In [46]:
labels = ["None", "Minimal", "Basic", "Good", "Excellent"]

save_text_label(all_records.iloc[7185]["text"], labels[4])

In [53]:
pd.read_csv("fineweb2_data.csv")  # Load the saved CSV to check the saved labels

,text,label
0,"Du kan bruge visningstilstande, når du vil læse dokumentet eller flytte rundt i det.\nHvis du vil læse dokumentet uden at anstrenge øjnene mere end højst nødvendigt og bruge værktøjer, der er specielt beregnet til læsning af dokumenter, skal du vælge læselayout (læselayout: En visning, der er beregnet til læsning af dokumenter på en computerskærm. Dokumentet får en størrelse, der passer til skærmen, de fleste værktøjslinjer er fjernet, men der er stadig kommandoer til navigering, kommenterin...",['Good']
1,"\n\nDen indre barriere…\nPerspektiv Nr. 2-2000\nSkær ned på fedtet – spis flere kulhydrater – men hvilke?\nTatjana Hejgaard Vasilaras, M. Sc., Ph.D. stud. & Arne Astrup, professor dr.med., Forskningsinstitut for Human Ernæring.\nAnbefalingerne er klare: Vi skal skære ned på fedtet og erstatte det med fødevarer, der har et højere indhold af kulhydrat og protein.\nVæk med piskefløden, osten, de fede kød- og pålægsprodukter, chipsene og fedtstofferne. Ind med brød, pasta, ris, grøntsager og de ...",['Good']
2,"\n\nDe hører ligesom Cirrus og Cirrostratus til de høje skyer, hvilket vil sige, at de typisk befinder sig i tre til otte kilometers højde i polare områder, fem til 13 kilometers højde i tempererede områder og seks til 18 kilometer i tropiske områder.\nCirrocumulus kan dannes i klar luft og kan være resultat af omdannelsen af Cirrus- eller Cirrostratusskyerne. De kan også dannes som resultat af formindskelse af de elementer som en Altocumulusflage eller et lag består af.",['Good']
3,"Pisk æggehvider helt stive og vend forsigtigt de hakkede mandler, flormelis og essens heri.\nSæt dejen i små skefulde på en plade med bagepapir, der skal være god afstand mellem kagerne.\nPynt med en halv mandel på hver og bag dem ved 180 grader i ca. 12 minutter - til de er svagt lysebrune. Køl dem af på en rist.",['Good']
4,"Rens din skærm billigst og bedst\nHvordan renser man mest effektivt sin computer- eller tv-skærm? Fotograf og forfatter til blandt andet Politikens store bog om digital fotografering Thomas Nykrog har undersøgt emnet.\nBomuld og lunkent vand\nHan har blandt andet talt med mange computerfirmaer, og det viser sig, at en skærm renses bedst med en bomuldsklud, der ikke fnuller og almindeligt lunkent vand. De firmaer som selv fabrikerer rensevæske til skærme, anbefaler selvfølgelig, at man bruger...",['Good']
5,"Pil løgene og hak dem fint. Skyl bladselleristænglerne og skær dem ud i mindre stykker. Steg løgene klare i smør i en gryde med bladselleri, tilsæt skyllede timianblade eller tørret timian, paprika og majskerner.\nHæld mælk i og kog suppen ca. 15 min. Tilsæt salt. Blend suppen, til den er helt jævn og glat (kom den evt. igennem en grov si).\nSkær kyllingekødet i små tern. Kom kyllingekødet i den kogende suppe, og skru ned, så suppen småkoger i ca. 10 min., til kyllingekødet er gennemkogt.\nT...",['Good']
6,"Tomaterne skoldes, flås og skæres i små tern (fjern også de fleste kerner). De hakkede tomater puttes i en gryde og bringes i kog sammen med bouillon, basilikum eller timian, laurbærbladet, salt, peber og chili. Suppen bringes i kog og småkoger ca. 15 min. Herefter fjernes basilikum/timian og laurbærbladet og suppen pureres med en stavblender eller i en blender/food-processor. Put Creme Fine og kødboller i den purerede suppe, varm den op og smag evt. til med krydderier.\nServeres med et par ...",['Good']
7,"Varm ovnen op til 200o . Udrør gæren med vand. Tilsæt olie, salt og 2/3 af melet. Når melet slipper skålen vendes dejen ud på bordet og æltes igennem med mere mel.\nSæt dejen til hævning dækket med et fugtigt klæde 20 min.\nÆlt dejen igennem. Del den i to. Rul dem meget tyndt ud på et meldrysset bord.\nLæg bundene på plade beklædt med bagepapir og prik dem med en gaffel.\nPensl kanterne med lidt olie. Forbag dem uden fyld 10-15 min.\nRiv alle grøntsager fint evt. med foodprocessor.\nSvits gr...",['Good']
8,"Anden renses for indmad og skylles. Gnides herefter godt med salt og peber indvendig og u